In [ ]:
from google.cloud import translate_v3 as translate
import os

## Google Cloud API 정보 입력

In [ ]:
project_id = "your_project_id" ## GCP project id 
glossary_id = "your_glossary_id" ## name you want to give this glossary resource
location = 'us-central1'
credential_file = "/path/to/your/credentials.json"
gcs_glossary_uri = 'gs://path/to/your/storage/your_glossary.csv'

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=credential_file

## 용어집 삭제(필요시 사용)

In [ ]:
def delete_glossary(
    project_id, glossary_id
):
    client = translate.TranslationServiceClient()

    parent = client.glossary_path(project_id, "us-central1", glossary_id)

    operation = client.delete_glossary(parent)
    result = operation.result(timeout=180)

    print("Deleted: {}".format(result.name))

In [ ]:
# delete_glossary(project_id, glossary_id)

## 용어집 리소스 생성

In [ ]:
def create_glossary(
    project_id,
    input_uri,
    glossary_id
):
    """
    Create a equivalent term sets glossary. Glossary can be words or
    short phrases (usually fewer than five words).
    https://cloud.google.com/translate/docs/advanced/glossary#format-glossary
    """
    client = translate.TranslationServiceClient()

    # Supported language codes: https://cloud.google.com/translate/docs/languages
    source_lang_code = "ko"
    target_lang_code = "en"
    location = "us-central1"  # The location of the glossary

    name = client.glossary_path(project_id, location, glossary_id)
    language_codes_set = translate.types.Glossary.LanguageCodesSet(
        language_codes=[source_lang_code, target_lang_code]
    )

    gcs_source = translate.types.GcsSource(input_uri=input_uri)

    input_config = translate.types.GlossaryInputConfig(gcs_source=gcs_source)

    glossary = translate.types.Glossary(
        name=name, language_codes_set=language_codes_set, input_config=input_config
    )

    parent = client.location_path(project_id, location)
    # glossary is a custom dictionary Translation API uses
    # to translate the domain-specific terminology.
    operation = client.create_glossary(parent=parent, glossary=glossary)

    result = operation.result(timeout=180)
    print("Created: {}".format(result.name))
    print("Input Uri: {}".format(result.input_config.gcs_source.input_uri))

In [ ]:
create_glossary(project_id,gcs_glossary_uri,glossary_id)

## 용어집 정보 조회

In [ ]:
from google.cloud import translate_v3 as translate
client = translate.TranslationServiceClient()

parent = client.glossary_path(
    project_id,
    location,  # The location of the glossary
    glossary_id)

response = client.get_glossary(parent)
print('Name: {}'.format(response.name))
print('Language Pair:')
print('\tSource Language Code: {}'.format(
    response.language_pair.source_language_code))
print('\tTarget Language Code: {}'.format(
    response.language_pair.target_language_code))
print('Input Uri: {}'.format(
    response.input_config.gcs_source.input_uri))